In [1]:
# importing the required library
import json
import hashlib
import copy
from datetime import datetime, timezone, timedelta

In [2]:
# defining the class
class SimpleBlockchain:
    def __init__(self, difficulty=4):
        """
        Initializes the blockchain with a specified difficulty.
        Automatically creates the genesis block.
        """
        self.chain = []               # The list storing the entire blockchain
        self.data_set = set()         # Used to prevent duplicate data entries
        self.difficulty = difficulty  # Proof-of-work difficulty level
        self.create_genesis_block()   # Create the first block in the chain

    def create_genesis_block(self):
        """
        Creates the first block in the blockchain with predefined values.
        This block has no previous hash and serves as the foundation.
        """
        genesis_block = {
            'index': 0,
            'timestamp': datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S"),
            'data': "Genesis Block",
            'previous_hash': '0',
            'nonce': 0,
            'hash': ''
        }
        genesis_block['hash'] = self.compute_hash(genesis_block)
        self.chain.append(genesis_block)
        self.data_set.add("Genesis Block")

    def add_block(self, data):
        """
        Adds a new block with the given data to the blockchain,
        after verifying the data is valid and unique.
        """
        if not data:
            return None  # Reject empty data

        if data in self.data_set and data != "Genesis Block":
            return None  # Reject duplicates

        last_block = self.chain[-1]
        new_block = {
            'index': len(self.chain),
            'timestamp': datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S"),
            'data': data,
            'previous_hash': last_block['hash'],
            'nonce': 0,
            'hash': ''
        }

        new_block['hash'] = self.proof_of_work(new_block)
        self.chain.append(new_block)
        self.data_set.add(data)
        return new_block

    def compute_hash(self, block):
        """
        Computes a SHA-256 hash of a block, excluding the 'hash' field.
        """
        block_copy = block.copy()
        block_copy.pop('hash', None)
        encoded_block = json.dumps(block_copy, sort_keys=True).encode()
        return hashlib.sha256(encoded_block).hexdigest()

    def proof_of_work(self, block):
        """
        Performs the proof-of-work algorithm by finding a nonce such that the
        block's hash starts with a number of leading zeros defined by difficulty.
        """
        block['nonce'] = 0
        while True:
            block['hash'] = self.compute_hash(block)
            if block['hash'].startswith('0' * self.difficulty):
                return block['hash']
            block['nonce'] += 1

    def is_chain_valid(self, chain):
        """
        Validates the integrity of the blockchain by checking hashes,
        previous hash links, and proof-of-work conditions.
        """
        for i in range(1, len(chain)):
            current = chain[i]
            previous = chain[i - 1]

            if current['previous_hash'] != previous['hash']:
                print(f"Validation Error: Block {current['index']} previous hash mismatch.")
                return False

            computed_hash = self.compute_hash(current.copy())
            if current['hash'] != computed_hash:
                print(f"Validation Error: Block {current['index']} hash mismatch.")
                return False

            if not current['hash'].startswith('0' * self.difficulty):
                print(f"Validation Error: Block {current['index']} does not meet PoW difficulty.")
                return False

        return True

    def replace_chain(self, new_chain):
        """
        Replaces the current chain with a new, longer, valid chain.
        This supports the longest-chain consensus protocol.
        """
        if len(new_chain) > len(self.chain) and self.is_chain_valid(new_chain):
            self.chain = copy.deepcopy(new_chain)
            self.data_set = {block['data'] for block in self.chain}
            print("Chain replaced ✅ - New chain is longer and valid.")
            return True
        elif len(new_chain) <= len(self.chain):
            print("Chain not replaced ❌ - New chain is not longer than current chain.")
        else:
            print("Chain not replaced ❌ - New chain is invalid.")
        return False

    def export_chain(self, filename="blockchain.json"):
        """
        Exports the blockchain to a JSON file for persistence or sharing.
        """
        with open(filename, "w") as f:
            json.dump(self.chain, f, indent=4)
        print(f"Blockchain exported to '{filename}'.")


In [3]:
# Test 1: Initialize the blockchain
blockchain = SimpleBlockchain()
print("Test 1: Blockchain initialized. Genesis block created.")
print("Current Chain Length:", len(blockchain.chain))
print(f"Genesis Block Details:\n{json.dumps(blockchain.chain[0], indent=4)}")
print("-" * 60)

Test 1: Blockchain initialized. Genesis block created.
Current Chain Length: 1
Genesis Block Details:
{
    "index": 0,
    "timestamp": "2025-06-09 14:55:45",
    "data": "Genesis Block",
    "previous_hash": "0",
    "nonce": 0,
    "hash": "b02158d31d77d37a79b45f267118d267fd49f1188c820f57a21e9484da48e6c5"
}
------------------------------------------------------------


In [4]:
# Test 2: Try adding a valid block with unique data
print("Test 2: Adding valid block with unique data.")
data_to_add_1 = "Patient A - Diagnosis: Flu"
result = blockchain.add_block(data_to_add_1)
if result:
    print(f"Result: Added ✅ - Data '{data_to_add_1}' is unique and valid.")
    print(f"Block {blockchain.chain[-1]['index']} Details:\n{json.dumps(blockchain.chain[-1], indent=4)}")
else:
    print(f"Result: Rejected ❌ - Could not add '{data_to_add_1}'.")
print("-" * 60)

Test 2: Adding valid block with unique data.
Result: Added ✅ - Data 'Patient A - Diagnosis: Flu' is unique and valid.
Block 1 Details:
{
    "index": 1,
    "timestamp": "2025-06-09 14:55:45",
    "data": "Patient A - Diagnosis: Flu",
    "previous_hash": "b02158d31d77d37a79b45f267118d267fd49f1188c820f57a21e9484da48e6c5",
    "nonce": 155409,
    "hash": "0000587b698a57b2b6ec0044fa40505745c7b893cc758170b1952d40ebd41b22"
}
------------------------------------------------------------


In [5]:
# Test 3: Try adding another valid block with different data
print("Test 3: Adding another valid block with different data.")
data_to_add_2 = "Patient B - Diagnosis: Cold"
result = blockchain.add_block(data_to_add_2)
if result:
    print(f"Result: Added ✅ - Data '{data_to_add_2}' is unique and valid.")
    print(f"Block {blockchain.chain[-1]['index']} Details:\n{json.dumps(blockchain.chain[-1], indent=4)}")
else:
    print(f"Result: Rejected ❌ - Could not add '{data_to_add_2}'.")
print("-" * 60)

Test 3: Adding another valid block with different data.
Result: Added ✅ - Data 'Patient B - Diagnosis: Cold' is unique and valid.
Block 2 Details:
{
    "index": 2,
    "timestamp": "2025-06-09 14:55:49",
    "data": "Patient B - Diagnosis: Cold",
    "previous_hash": "0000587b698a57b2b6ec0044fa40505745c7b893cc758170b1952d40ebd41b22",
    "nonce": 2479,
    "hash": "0000ca2e1131e81979047eea84453c8b287c773e4a713c445dbae70ac4ca22f3"
}
------------------------------------------------------------


In [6]:
# Test 4: Try adding a block with empty data
print("Test 4: Attempting to add block with empty data.")
data_to_add_empty = ""
result = blockchain.add_block(data_to_add_empty)
if result:
    print(f"Result: Added ✅ - Data '{data_to_add_empty}' is unique and valid.")
    print(f"Block {blockchain.chain[-1]['index']} Details:\n{json.dumps(blockchain.chain[-1], indent=4)}")
else:
    print(f"Result: Rejected ❌ - Empty data not allowed.")
print("-" * 60)

Test 4: Attempting to add block with empty data.
Result: Rejected ❌ - Empty data not allowed.
------------------------------------------------------------


In [7]:
# Test 5: Try adding a duplicate block
print("Test 5: Attempting to add duplicate data block.")
data_to_add_duplicate = "Patient A - Diagnosis: Flu" # This was added in Test 2
result = blockchain.add_block(data_to_add_duplicate)
if result:
    print(f"Result: Added ✅ - Data '{data_to_add_duplicate}' is unique and valid.")
    print(f"Block {blockchain.chain[-1]['index']} Details:\n{json.dumps(blockchain.chain[-1], indent=4)}")
else:
    print(f"Result: Rejected ❌ - Duplicate data '{data_to_add_duplicate}' already exists.")
print("-" * 60)

Test 5: Attempting to add duplicate data block.
Result: Rejected ❌ - Duplicate data 'Patient A - Diagnosis: Flu' already exists.
------------------------------------------------------------


In [8]:
# Test 6: Print full chain contents
print("Test 6: Printing current blockchain contents.")
for block in blockchain.chain:
    print(f"Index: {block['index']}, Data: {block['data']}, Hash: {block['hash'][:10]}...") # Truncate hash for readability
print("-" * 60)

Test 6: Printing current blockchain contents.
Index: 0, Data: Genesis Block, Hash: b02158d31d...
Index: 1, Data: Patient A - Diagnosis: Flu, Hash: 0000587b69...
Index: 2, Data: Patient B - Diagnosis: Cold, Hash: 0000ca2e11...
------------------------------------------------------------


In [9]:
# Test 7: Validate the chain
print("Test 7: Validating blockchain integrity.")
is_valid = blockchain.is_chain_valid(blockchain.chain)
print("Chain is valid ✅" if is_valid else "Chain is NOT valid ❌ - See errors above.")
print("-" * 60)

Test 7: Validating blockchain integrity.
Chain is valid ✅
------------------------------------------------------------


In [10]:
# Test 8: Tamper with a block to simulate attack
print("Test 8: Tampering with data in Block 1 to simulate attack.")
tampered_chain = copy.deepcopy(blockchain.chain)
original_data_block_1 = tampered_chain[1]['data']
tampered_chain[1]['data'] = "Patient A - HACKED DIAGNOSIS!"
print(f"Tampered Block {tampered_chain[1]['index']} data from '{original_data_block_1}' to '{tampered_chain[1]['data']}'.")
print("-" * 60)

Test 8: Tampering with data in Block 1 to simulate attack.
Tampered Block 1 data from 'Patient A - Diagnosis: Flu' to 'Patient A - HACKED DIAGNOSIS!'.
------------------------------------------------------------


In [11]:
# Test 9: Validate the chain
print("Test 9: Validating blockchain integrity.")
is_valid = blockchain.is_chain_valid(blockchain.chain)
print("Chain is valid ✅" if is_valid else "Chain is NOT valid ❌ - See errors above.")
print("-" * 60)

Test 9: Validating blockchain integrity.
Chain is valid ✅
------------------------------------------------------------


In [12]:
# Test 10: Revalidate the chain after tampering
print("Test 10: Revalidating chain after tampering (using the tampered copy).")
is_valid_after_tamper = blockchain.is_chain_valid(tampered_chain)
print("Chain is valid ✅" if is_valid_after_tamper else "Chain is NOT valid ❌ - Tampering detected as expected.")
print("Note: Tampering with block data changes its hash, breaking the chain link and PoW.")
print("-" * 60)

Test 10: Revalidating chain after tampering (using the tampered copy).
Validation Error: Block 1 hash mismatch.
Chain is NOT valid ❌ - Tampering detected as expected.
Note: Tampering with block data changes its hash, breaking the chain link and PoW.
------------------------------------------------------------


In [13]:
# Test 11: Simulate a longer, valid chain from another node
print("Test 11: Simulating a longer, valid chain from another node.")
another_node_blockchain = SimpleBlockchain()
another_node_blockchain.add_block("Patient C - Diagnosis: Asthma (from another node)")
another_node_blockchain.add_block("Patient D - Diagnosis: Allergies (from another node)")
another_node_blockchain.add_block("Patient E - Diagnosis: Migraine (from another node)")
assert another_node_blockchain.is_chain_valid(another_node_blockchain.chain)
print(f"Simulated new chain length: {len(another_node_blockchain.chain)}")
print(f"Current blockchain length: {len(blockchain.chain)}")
print("-" * 60)

Test 11: Simulating a longer, valid chain from another node.
Simulated new chain length: 4
Current blockchain length: 3
------------------------------------------------------------


In [14]:
# Test 12: Apply consensus to adopt longer valid chain
print("Test 12: Applying consensus: attempting to replace current chain with the new longer valid chain.")
result_consensus = blockchain.replace_chain(another_node_blockchain.chain)
if result_consensus:
    print("Consensus result: Chain replaced ✅ - Adopted the longer, valid chain.")
    print("Details of the newly adopted chain:")
    for block in blockchain.chain:
        print(f"  Index: {block['index']}, Data: {block['data']}, Hash: {block['hash'][:10]}...") # Truncate hash for readability
else:
    print("Consensus result: Chain NOT replaced ❌ - Either shorter or invalid.")
print(f"New Current Chain Length: {len(blockchain.chain)}")
print("-" * 60)

Test 12: Applying consensus: attempting to replace current chain with the new longer valid chain.
Chain replaced ✅ - New chain is longer and valid.
Consensus result: Chain replaced ✅ - Adopted the longer, valid chain.
Details of the newly adopted chain:
  Index: 0, Data: Genesis Block, Hash: f8e59c59b2...
  Index: 1, Data: Patient C - Diagnosis: Asthma (from another node), Hash: 0000a9beb7...
  Index: 2, Data: Patient D - Diagnosis: Allergies (from another node), Hash: 0000156383...
  Index: 3, Data: Patient E - Diagnosis: Migraine (from another node), Hash: 0000482708...
New Current Chain Length: 4
------------------------------------------------------------


In [15]:
# Test 13: Print final chain contents
print("Test 13: Final blockchain contents after consensus.")
for block in blockchain.chain:
    print(f"Index: {block['index']}, Data: {block['data']}, Hash: {block['hash'][:10]}...") # Truncate hash for readability
print("\n✅  Final blockchain.")

Test 13: Final blockchain contents after consensus.
Index: 0, Data: Genesis Block, Hash: f8e59c59b2...
Index: 1, Data: Patient C - Diagnosis: Asthma (from another node), Hash: 0000a9beb7...
Index: 2, Data: Patient D - Diagnosis: Allergies (from another node), Hash: 0000156383...
Index: 3, Data: Patient E - Diagnosis: Migraine (from another node), Hash: 0000482708...

✅  Final blockchain.


In [16]:
# Test 14 Test for nonce reuse

nonces = [block['nonce'] for block in blockchain.chain]
assert len(nonces) == len(set(nonces)), "❌ Nonces reused across blocks!"
print("✅ Test 14 Nonce uniqueness confirmed.")


✅ Test 14 Nonce uniqueness confirmed.


In [17]:
# list of all blocks
blockchain.chain

[{'index': 0,
  'timestamp': '2025-06-09 14:55:50',
  'data': 'Genesis Block',
  'previous_hash': '0',
  'nonce': 0,
  'hash': 'f8e59c59b2df9ad9a28820cb32fb96d272b5d583d6a54ba28a392d81eb07d53d'},
 {'index': 1,
  'timestamp': '2025-06-09 14:55:50',
  'data': 'Patient C - Diagnosis: Asthma (from another node)',
  'previous_hash': 'f8e59c59b2df9ad9a28820cb32fb96d272b5d583d6a54ba28a392d81eb07d53d',
  'nonce': 74951,
  'hash': '0000a9beb725b65f5648cc6813fae3362f7f3a48afbdb8d89b6af89f8fd7c8ac'},
 {'index': 2,
  'timestamp': '2025-06-09 14:55:51',
  'data': 'Patient D - Diagnosis: Allergies (from another node)',
  'previous_hash': '0000a9beb725b65f5648cc6813fae3362f7f3a48afbdb8d89b6af89f8fd7c8ac',
  'nonce': 9175,
  'hash': '00001563836235960e107f9993d744b9e85167e737eee7098fbc84e4cc17f298'},
 {'index': 3,
  'timestamp': '2025-06-09 14:55:52',
  'data': 'Patient E - Diagnosis: Migraine (from another node)',
  'previous_hash': '00001563836235960e107f9993d744b9e85167e737eee7098fbc84e4cc17f298',
